In [ ]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import os
import add_funcs_231023 as af
from scipy import integrate
from astropy.io import ascii

In [ ]:
#File paths
base_dir = '/home/stba7609/'
obs_dir = base_dir + 'SECRETO/WISEREP_IIb/Standardised_spectra/'

fnames = sorted(os.listdir(obs_dir))

In [ ]:
#Read in all observations from SN2011dh
wl_list, flux_list = [], []
epochs = []

for i in range(len(fnames)):
    if '2011dh' in fnames[i]:
    
        spectrum = np.loadtxt(obs_dir + fnames[i] , delimiter = ',')

        wl_left, wl_right = 5000, 8000
        zoom_mask = (spectrum[:, 0] > wl_left) * (spectrum[:, 1] < wl_right)

        wl_zoom, spec_zoom = spectrum[:, 0][zoom_mask], spectrum[:, 1][zoom_mask]

        wl_list.append(wl_zoom)
        flux_list.append(spec_zoom)
        epochs.append(int(fnames[i][-7:-4])) #Store epochs to later select a subset for plotting
        


In [ ]:
#Get the best fit params for all obs of SN2011dh

popt_best, pcov_best, score_g, tau_best, g_check = af.simultaneous_fitting_g(wl_list, flux_list, epochs, plot = True)
if g_check == True: #The velocity for the Gaussian NII is accepted, so we continue with a gaussian
    shape = 'g'
    pass
elif g_check == False: #The velocity is too high, meaning a tophat is a better fit
    popt_best, pcov_best, score_th, tau_best = af.simultaneous_fitting_th(wl_list, flux_list, epochs, plot = True)
    shape = 'th'
    print('The scores are for g and th: ', score_g, score_th, tau_best)

In [ ]:
#From the best fit params, get the NII amount

fitted_NII, fitted_sigma = [], []
for k in range(len(epochs)):
    wl, flux = wl_list[k], flux_list[k]
    popt, pcov = popt_best[k], pcov_best[k]
    integrated_NII_flux_my_method, sigma_NII = af.observed_flux_from_fit(wl, flux, popt, pcov, wl_left, wl_right, shape)

    fitted_NII.append(integrated_NII_flux_my_method*100)
    fitted_sigma.append(sigma_NII*100)
    print("The NII doublet carries: ", "{0:.3g}".format(integrated_NII_flux_my_method*100), "% of the selected region")


In [ ]:
#Plot a subset of epochs
plot_epochs = np.array([155, 201, 239, 292, 359, 415])


fig, ax = plt.subplots(2, 3, figsize =  (10, 5), sharex = True, sharey = True)
                       
for i in range(len(plot_epochs)):
                       
    index = np.where(plot_epochs[i] == np.array(epochs))[0][0]
    wl_data_zoom, flux_data_zoom = wl_list[index], flux_list[index]
    
    OI_region = (wl_data_zoom > 6100) * (wl_data_zoom < 6900)
    normalisation = np.max(flux_data_zoom[OI_region])

    ax[int(i/3), i%3].plot(wl_data_zoom, flux_data_zoom/normalisation, c = 'k', alpha = 0.3)
    
    #Also plot the continuum removed flux
    flux_data_zoom_corr = af.continuum_remover(wl_data_zoom, flux_data_zoom)
    ax[int(i/3), i%3].plot(wl_data_zoom, flux_data_zoom_corr/normalisation, c = 'green')
    
    
    if tau_best[index] < 0.1:
        ax[int(i/3), i%3].plot(wl_data_zoom, af.gaussian(wl_data_zoom, *popt_best[index][::2])/normalisation, c = 'fuchsia', linestyle = '--')
        ax[int(i/3), i%3].plot(wl_data_zoom, af.gaussian(wl_data_zoom, *popt_best[index][::2])/normalisation + af.gaussian(wl_data_zoom, *popt_best[index][1::2])/normalisation, c = 'k', linestyle = '--')

    else:
        ax[int(i/3), i%3].plot(wl_data_zoom, af.escatt(wl_data_zoom, *popt_best[index][::2], tau_best[i])/normalisation, c = 'fuchsia', linestyle = '--')
        ax[int(i/3), i%3].plot(wl_data_zoom, af.escatt(wl_data_zoom, *popt_best[index][::2], tau_best[i])/normalisation + af.gaussian(wl_data_zoom, *popt_best[index][1::2])/normalisation, c = 'k', linestyle = '--')

    
    ax[int(i/3), i%3].plot(wl_data_zoom, af.gaussian(wl_data_zoom, *popt_best[index][1::2])/normalisation, c = 'gold', linestyle = '--')
    
    
    trans = ax[int(i/3), i%3].get_xaxis_transform()
    velocity_fwhm = abs(af.v_from_labda(popt_best[index][1]-popt_best[index][5], popt_best[index][1]))
    fig.text(6350, 0.7, s = '$V_{FWHM, NII}$ = ' + '{:.4g}'.format( np.sqrt(2*np.log(2)) *velocity_fwhm) + ' km/s', transform = trans)
    
    fig.text(6540, 0.8, s = 't = ' + '{:.3g}'.format(plot_epochs[i]) + 'd', transform = trans, c = 'red')
    
#To get the labels right, the following subsection
ax[0, 0].plot([0, 1], [0, 1], c = 'k', label = 'Observed Flux', alpha = 0.3)
ax[0, 0].plot([0, 1], [0, 1], c = 'green', label = 'Continuum Corrected Flux')
ax[0, 0].plot([0, 1], [0, 1], c = 'k', label = 'Total Fit', linestyle = '--')
ax[0, 0].plot([0, 1], [0, 1], c = 'fuchsia', label = 'OI Fit', linestyle = '--')
ax[0, 0].plot([0, 1], [0, 1], c = 'gold', label = 'NII Fit', linestyle = '--')


ax[0, 0].set_xlim(6100, 6900)
ax[0, 0].set_ylim(0, 1)

fig.text(0.42, 0.02, s = 'Rest Wavelength [Å]', fontsize = 14, c = 'black')
fig.text(0.07, 0.36, s = 'Normalised flux', fontsize = 14, c = 'black', rotation = 90)

plt.figlegend(bbox_to_anchor=(0.03, 0.93, 1, 0.0), loc= 'center',
              borderaxespad = 0, handletextpad = 0.3, ncol=10 ,columnspacing=1.6, frameon= False)

fig.savefig('Figure_5.pdf', dpi = 300, bbox_inches = 'tight')
plt.show()